# 1. Connecting to Azure and getting the resources from correct Resource Group 

**Before you start you need to login to Azure to correct subscription**

Log into Azure using the Azure CLI:
```powershell
az login --tenant ''
az account set --subscription ''
```

**Setup some variables from Azure that will be used in the code later**

In [ ]:
%pip install python-dotenv

In [1]:


from dotenv import load_dotenv
import os
# Load environment variables from .env file
load_dotenv()

# Assign variables from environment variables
subscription_id = os.getenv('AZURE_SUBSCRIPTION_ID')
resource_group_name = os.getenv('AZURE_RESOURCE_GROUP_NAME')
connection_string = os.getenv('AZURE_STORAGE_CONNECTION_STRING')
container_name = os.getenv('AZURE_CONTAINER_NAME')
storage_account_name = os.getenv('AZURE_STORAGE_ACCOUNT_NAME')

*GitHub Copilot is great with help to install as well the libraries that are missing in the code.*

*To do that ask Copilot to **/fix** the code and you will get the code as below*

*Sometimes Copilot will show code like ```python !pip``` instead of ```python %pip```, but the code will still works for you, as the model is not updated yet with 100% of latest changes of libraries*

In [ ]:
%pip install azure-identity
%pip install azure-mgmt-resource

**GitHub Copilot**

Connect to Azure using Azure Identity and list all the resources from provided resource group. Save the resourceId to the array called 'resource_array'

In [9]:

from azure.identity import DefaultAzureCredential
from azure.mgmt.resource import ResourceManagementClient

# Authenticate using DefaultAzureCredential
credential = DefaultAzureCredential()

# Initialize the ResourceManagementClient
resource_client = ResourceManagementClient(credential, subscription_id)

# List all resources in the specified resource group
resources = resource_client.resources.list_by_resource_group(resource_group_name)

# Save the resource IDs to the array
resource_array = [resource.id for resource in resources]

**GitHub Copilot**

For each resource.id in resource_array, get the json from Azure and save it as a separate file to the folder json_files

In [13]:
import os
import json

# Ensure the directory exists
os.makedirs('json_files', exist_ok=True)

# Iterate over each resource ID in the resource_array
for resource_id in resource_array:
    # Get the resource details
    if 'servers' in resource_id:
        api_version = '2014-01-01'
    else:
        if 'storageAccounts' in resource_id:
            api_version = '2024-01-01'
        else:
            api_version = '2023-12-01'
    resource = resource_client.resources.get_by_id(resource_id, api_version=api_version)
    
    # Convert the resource details to a dictionary
    resource_json = resource.as_dict()
    
    # Create a file name based on the resource name
    file_name = f"json_files/{resource.name}.json"
    
    # Save the resource details to a JSON file
    with open(file_name, 'w') as f:
        json.dump(resource_json, f, indent=4)

print("Resource details saved to json_files directory.")

Resource details saved to json_files directory.


**GitHub Copilot**

If the resource type is 'servers', use api_version: '2014-01-01', otherwise use api_version '2023-12-01'

# 2. Save the files to the blob storage

**GitHub Copilot**

Save all the files from folder json_files to the provided blob storage

In [ ]:
%pip install azure.storage

In [ ]:
# Save all the files from folder json_files to the provided blob storage

from azure.storage.blob import BlobServiceClient

# Initialize the BlobServiceClient
blob_service_client = BlobServiceClient.from_connection_string(connection_string)

container_client = blob_service_client.get_container_client(container_name)

# Iterate over each file in the json_files directory
for file_name in os.listdir('json_files'):
    file_path = os.path.join('json_files', file_name)
    
    # Upload the file to the blob storage
    with open(file_path, 'rb') as data:
        container_client.upload_blob(name=file_name, data=data, overwrite=True)

print("All files have been uploaded to the blob storage.")

# 3. Create a mermaid diagram

Install the required libraries

In [ ]:
%pip install mermaid-py

**GitHub Copilot**

Use the mermaid-py library and 'from mermaid.graph import Graph' to create a graph from files in folder json_files

In [ ]:
from mermaid.graph import Graph
import os
import json

# Initialize a new graph
graph = Graph()

# Iterate over each file in the json_files directory
for file_name in os.listdir('json_files'):
    file_path = os.path.join('json_files', file_name)
    
    # Load the JSON data from the file
    with open(file_path, 'r') as f:
        resource_data = json.load(f)
    
    # Add nodes and edges to the graph based on the resource data
    resource_id = resource_data.get('id')
    resource_name = resource_data.get('name')
    resource_type = resource_data.get('type')
    
    # Add the resource as a node
    graph.add_node(resource_id, label=resource_name, shape='box')
    
    # Add edges based on relationships (if any)
    # This is a placeholder for adding edges, modify as needed based on your data structure
    if 'dependencies' in resource_data:
        for dependency in resource_data['dependencies']:
            graph.add_edge(resource_id, dependency)

# Save the graph to a markdown file
with open('diagram.md', 'w') as f:
    f.write(graph.to_markdown())

print("Graph has been created and saved to diagram.md.")

*Check the example of using the mermaid library*

**GitHub Copilot**

From the each file in folder json_files, get the id and based on that generate a markdown file, print it and save to the diagram.md file

In [ ]:
import os
import json

# Initialize a new markdown content
markdown_content = "# Resource Diagram\n\n"

# Iterate over each file in the json_files directory
for file_name in os.listdir('json_files'):
    file_path = os.path.join('json_files', file_name)
    
    # Load the JSON data from the file
    with open(file_path, 'r') as f:
        resource_data = json.load(f)
    
    # Get the resource id
    resource_id = resource_data.get('id')
    
    # Append the resource id to the markdown content
    markdown_content += f"- **Resource ID**: {resource_id}\n"

# Save the markdown content to a file
with open('diagram.md', 'w') as f:
    f.write(markdown_content)

# Print the markdown content
print(markdown_content)

In [ ]:
from mermaid.graph import Graph
import os

# Initialize a new graph
graph = Graph()

# Ensure the diagram.md file exists
if os.path.exists('diagram.md'):
    # Read the content of the diagram.md file
    with open('diagram.md', 'r') as f:
        lines = f.readlines()

    # Iterate over each line in the file
    for line in lines:
        if line.startswith('- **Resource ID**:'):
            resource_id = line.split(': ')[1].strip()
            resource_name = resource_id.split('/')[-1]
            
            # Add the resource as a node
            graph.add_node(resource_id, label=resource_name, shape='box')

    # Save the graph to a markdown file
    with open('mermaid_diagram.md', 'w') as f:
        f.write(graph.to_markdown())

    print("Mermaid diagram has been created and saved to mermaid_diagram.md.")
else:
    print("The file diagram.md does not exist.")